In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [157]:
#df = pd.read_csv('../elena/barros_2011_training.csv')
df = pd.read_csv('../elena/data/training_final_latcorr.csv')

## Clean / Reverse Engineer Features
- Electricity generated (kWh)
- Drop redundant columns
- Drop non-lc columns

In [158]:
df['kWh'] = df['Area_km2'] / df['Area / Electricity']
df = df.drop(['Longitude', 'Latitude','CO2 (g/kWh)', 'CH4 (g/kWh)', 'Area / Electricity', 'Name', 'field_1',
              'temp_spring_avg', 'temp_summer_avg', 'temp_fall_avg', 'temp_winter_avg', 
              'NDVI_spring_avg', 'NDVI_summer_avg', 'NDVI_fall_avg', 'NDVI_winter_avg', 
              'npp_spring_avg' , 'npp_summer_avg' , 'npp_fall_avg' , 'npp_winter_avg'], axis=1)

In [159]:
df.columns

Index(['CO2 (mg C m¯² d¯¹)', 'CH4 (mg C m-2 d-1)', 'Area_km2', 'Age',
       'Volume_km3', 'Areakm2_div_Volkm3', 'org_c', 'temp_annual_avg',
       'temp_spring_avg_lc', 'temp_summer_avg_lc', 'temp_fall_avg_lc',
       'temp_winter_avg_lc', 'NDVI_annual_avg', 'NDVI_spring_avg_lc',
       'NDVI_summer_avg_lc', 'NDVI_fall_avg_lc', 'NDVI_winter_avg_lc',
       'npp_annual_avg', 'npp_spring_avg_lc', 'npp_summer_avg_lc',
       'npp_fall_avg_lc', 'npp_winter_avg_lc', 'erosion', 'kWh'],
      dtype='object')

In [160]:
df.shape

(154, 24)

In [ ]:
#create co2 df

In [161]:
co2 = df.drop(['CH4 (mg C m-2 d-1)'], axis=1)
co2 = co2[co2['CO2 (mg C m¯² d¯¹)'].notna()]

### Pre-processing

#### knn impute missing values

In [162]:
from sklearn.impute import KNNImputer
model_impute = KNNImputer(n_neighbors=int(np.sqrt(co2.shape[0])))
co2_imputed = model_impute.fit_transform(co2)

In [163]:
co2_imputed = pd.DataFrame(columns=co2.columns, data=co2_imputed)

In [164]:
#check
for column in co2_imputed:
    print(column, ':',  co2_imputed[column].isna().sum())

CO2 (mg C m¯² d¯¹) : 0
Area_km2 : 0
Age : 0
Volume_km3 : 0
Areakm2_div_Volkm3 : 0
org_c : 0
temp_annual_avg : 0
temp_spring_avg_lc : 0
temp_summer_avg_lc : 0
temp_fall_avg_lc : 0
temp_winter_avg_lc : 0
NDVI_annual_avg : 0
NDVI_spring_avg_lc : 0
NDVI_summer_avg_lc : 0
NDVI_fall_avg_lc : 0
NDVI_winter_avg_lc : 0
npp_annual_avg : 0
npp_spring_avg_lc : 0
npp_summer_avg_lc : 0
npp_fall_avg_lc : 0
npp_winter_avg_lc : 0
erosion : 0
kWh : 0


#### scaling features

In [165]:
from sklearn.preprocessing import StandardScaler
data = co2_imputed.drop(['CO2 (mg C m¯² d¯¹)'], axis=1).copy()
scaler = StandardScaler()
co2_scaled = pd.DataFrame(scaler.fit_transform(data),columns=co2.columns[1:])

In [166]:
co2_scaled.head()

,Area_km2,Age,Volume_km3,Areakm2_div_Volkm3,org_c,temp_annual_avg,temp_spring_avg_lc,temp_summer_avg_lc,temp_fall_avg_lc,temp_winter_avg_lc,...,NDVI_summer_avg_lc,NDVI_fall_avg_lc,NDVI_winter_avg_lc,npp_annual_avg,npp_spring_avg_lc,npp_summer_avg_lc,npp_fall_avg_lc,npp_winter_avg_lc,erosion,kWh
0,-0.637106,-0.913790,-0.488456,-0.363661,-0.555711,0.684757,0.777851,0.513276,0.615875,0.749077,...,0.739486,0.548385,0.941081,3.695553,3.309828,1.087479,3.217878,2.141876,1.884674,-0.022636
1,-0.637106,-0.913790,-0.488456,-0.363661,-0.555711,0.684757,0.777851,0.513276,0.615875,0.749077,...,0.739486,0.548385,0.941081,3.695553,3.309828,1.087479,3.217878,2.141876,1.884674,-0.022636
2,1.101332,-0.871820,4.377290,-0.426030,-0.657505,1.069519,1.054485,1.342929,1.045985,0.972523,...,1.251663,1.331399,1.399686,-0.347270,-0.013138,-1.220722,-0.072482,0.495821,3.018306,9.010330
3,-0.708398,2.527707,-0.584999,-0.242877,-0.274721,0.998409,0.964513,0.939314,1.014878,1.020211,...,0.708128,1.036800,0.992513,1.947225,1.844059,0.177579,2.455805,1.951695,-0.348399,-0.409070
4,-0.305322,0.303325,-0.449839,-0.165231,-0.830814,1.043744,1.069770,1.029762,1.007827,1.030781,...,-0.870217,-0.451701,0.071016,1.282326,0.361633,-0.357656,2.097183,1.368034,0.443458,-0.466019


## Support Vector Machine

In [167]:
X = co2_scaled
y = co2['CO2 (mg C m¯² d¯¹)']

In [168]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [169]:
from sklearn import svm

regr = svm.SVR()
regr.fit(x_train, y_train)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [170]:
print('train score:', regr.score(x_train, y_train))
print('test  score:', regr.score(x_test, y_test))

train score: -0.069229035617141
test  score: 0.006544677233467189


### linear kernel

In [171]:
regr2 = svm.SVR(kernel='linear')
regr2.fit(x_train, y_train)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [172]:
print('train score:', regr2.score(x_train, y_train))
print('test  score:', regr2.score(x_test, y_test))

train score: 0.12034293778309113
test  score: -0.04167438641993537


In [173]:
pd.DataFrame(columns=x_train.columns, data=regr2.coef_).T.sort_values(by=0)

,0
Age,-33.784747
npp_spring_avg_lc,-18.056804
Volume_km3,-15.942009
npp_summer_avg_lc,-15.745941
Areakm2_div_Volkm3,-12.205813
NDVI_spring_avg_lc,-3.844591
erosion,-1.091901
kWh,1.045439
NDVI_summer_avg_lc,1.152633
npp_annual_avg,1.522614


### scaling target variable:

In [174]:
y_train_scaled = scaler.fit_transform(np.array(y_train).reshape(-1,1))
y_test_scaled = scaler.fit_transform(np.array(y_test).reshape(-1,1))

In [175]:
regr_rbf_scaled = svm.SVR(kernel='rbf', C=10)
regr_rbf_scaled.fit(x_train, y_train_scaled)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR(C=10, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [176]:
print('train score:', regr_rbf_scaled.score(x_train, y_train_scaled))
print('test  score:', regr_rbf_scaled.score(x_test, y_test_scaled))

train score: 0.8882272968102802
test  score: 0.31334804160432994


### linear SVM & scaled target variable

In [177]:
regr4 = svm.SVR(kernel='linear')
regr4.fit(x_train, y_train_scaled)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [178]:
print('train score:', regr4.score(x_train, y_train_scaled))
print('test  score:', regr4.score(x_test, y_test_scaled))

train score: 0.36195698718903047
test  score: 0.15078547562485756


In [ ]:
#linear kernel reduces accuracy, as expected
#scaling target variable improved r2...lets now test out different kernels and params

### ploynomial kernel

In [179]:
regr_poly = svm.SVR(kernel='poly', C=12, degree=3)
regr_poly.fit(x_train, y_train_scaled)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR(C=12, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='poly', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [180]:
print('train score:', regr_poly.score(x_train, y_train_scaled))
print('test  score:', regr_poly.score(x_test, y_test_scaled))

train score: 0.8640441171460039
test  score: 0.305617410571498


In [ ]:
# tuning C increases the accuracy, likely because of significant overlap.
# C in the range of 10-20 seems to be most accurate without overfitting

### sigmoid kernel

In [181]:
regr_sigmoid = svm.SVR(kernel='sigmoid')
regr_sigmoid.fit(x_train, y_train_scaled)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='sigmoid', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [182]:
print('train score:', regr_sigmoid.score(x_train, y_train_scaled))
print('test  score:', regr_sigmoid.score(x_test, y_test_scaled))

train score: -1.143397687281352
test  score: -2.086415757309359


In [ ]:
#sigmoid kernel is not applicable

## Gridsearch to find best parameters

In [ ]:
#Next Steps:

#try gridsearch on rbf/poly to get better results
#parameters: C, gamma

In [220]:
from sklearn.model_selection import GridSearchCV

grid_values = {
    "C" : range(1,31),
    "gamma" : ['scale', 'auto']
}

grid_search_rbf_svm = GridSearchCV(regr_rbf_scaled, grid_values, scoring='r2', cv=None , n_jobs=-1)
%time grid_search_rbf_svm.fit(x_train, y_train_scaled)

CPU times: user 531 ms, sys: 30.5 ms, total: 562 ms
Wall time: 1.76 s


/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(cv=None, error_score=nan,
             estimator=SVR(C=10, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='scale', kernel='rbf',
                           max_iter=-1, shrinking=True, tol=0.001,
                           verbose=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': range(1, 31), 'gamma': ['scale', 'auto']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='r2', verbose=0)

In [221]:
print('best score :', grid_search_rbf_svm.best_score_)
print('best params:', grid_search_rbf_svm.best_params_)

best score : 0.32051049568590984
best params: {'C': 3, 'gamma': 'scale'}


In [222]:
pd.DataFrame(grid_search_rbf_svm.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.004988,0.000440,0.002136,0.000370,1,scale,"{'C': 1, 'gamma': 'scale'}",0.289224,0.335389,-0.101508,0.396173,0.243547,0.232565,0.174514,11
1,0.015350,0.017949,0.002886,0.001836,1,auto,"{'C': 1, 'gamma': 'auto'}",0.290091,0.333957,-0.105107,0.396733,0.245774,0.232289,0.175919,12
2,0.005969,0.001823,0.003159,0.000991,2,scale,"{'C': 2, 'gamma': 'scale'}",0.458076,0.345772,-0.063740,0.543655,0.238086,0.304370,0.210948,5
3,0.013845,0.015890,0.003127,0.001650,2,auto,"{'C': 2, 'gamma': 'auto'}",0.459764,0.345779,-0.067525,0.543887,0.234872,0.303356,0.212773,6
4,0.007572,0.002491,0.004949,0.003113,3,scale,"{'C': 3, 'gamma': 'scale'}",0.492543,0.352159,-0.010746,0.576781,0.191816,0.320510,0.210981,1
5,0.006409,0.000400,0.003317,0.002487,3,auto,"{'C': 3, 'gamma': 'auto'}",0.493468,0.351286,-0.016754,0.576602,0.187184,0.318357,0.213515,2
6,0.010385,0.005205,0.005751,0.006544,4,scale,"{'C': 4, 'gamma': 'scale'}",0.460175,0.368141,0.020967,0.533968,0.178277,0.312306,0.188127,3
7,0.009983,0.010613,0.002017,0.000314,4,auto,"{'C': 4, 'gamma': 'auto'}",0.458875,0.367266,0.023443,0.532468,0.171508,0.310712,0.187738,4
8,0.007129,0.001999,0.002412,0.000358,5,scale,"{'C': 5, 'gamma': 'scale'}",0.422933,0.374374,0.012807,0.482623,0.144000,0.287347,0.178891,7
9,0.011762,0.005573,0.003837,0.003678,5,auto,"{'C': 5, 'gamma': 'auto'}",0.422693,0.374773,0.022115,0.481170,0.128955,0.285941,0.178301,8


In [223]:
regr_rbf_scaled_best_params = svm.SVR(kernel='rbf', C=3, gamma='scale')
regr_rbf_scaled_best_params.fit(x_train, y_train_scaled)

print(regr_rbf_scaled_best_params.score(x_train, y_train_scaled))
print(regr_rbf_scaled_best_params.score(x_test, y_test_scaled))

0.7529269168378258
0.26967082521283026


/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
